In [100]:
import torch
from numpy import load
import pandas as pd
from torch import nn
import numpy as np

In [82]:
class MBTI_NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(4096, 16)
#         self.hidden2 = nn.Linear(1024, 512)
#         self.hidden3 = nn.Linear(512, 128)
#         self.output = nn.Linear(128, 16)
        
        # Define sigmoid activation and softmax output 
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        # Pass the input tensor through each of our operations
        x = self.hidden1(x)
        x = self.sigmoid(x)
#         x = self.hidden2(x)
#         x = self.sigmoid(x)
#         x = self.hidden3(x)
#         x = self.sigmoid(x)
#         x = self.output(x)
        x = self.softmax(x)
        
        return x

In [83]:
model = MBTI_NN()
model

MBTI_NN(
  (hidden1): Linear(in_features=4096, out_features=16, bias=True)
  (sigmoid): Sigmoid()
  (softmax): LogSoftmax()
)

In [3]:
df = pd.read_csv('/home/amoghmishra23/appledore/MBTI-Personality-Detection/data/mbti_comments/mbti_comments_cleaned.csv',chunksize=1000)
# embedding = load('output/embedding_whole.npy')

In [4]:
labels = []
for chunk in df:
    labels = labels + chunk['type'].tolist()

label_encoding = { "istj":1, "istp":2, "isfj":3, "isfp":4, "infj":5, "infp":6, "intj":7, "intp":8, "estp":9, "estj":10, "esfp":11, "esfj":12, "enfp":13, "enfj":14, "entp":15, "entj":16 }
label_enc = []
for label in labels:
    label_enc.append(label_encoding[label]-1)

In [14]:
with open('output/embedding_infersent.txt','r') as fr:
    embedding = fr.readlines()

In [10]:
embedding = list(map(eval, embedding)) 

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(embedding,label_enc,test_size=0.2,random_state=42)

In [22]:
Y_train

[5,
 14,
 3,
 5,
 5,
 5,
 7,
 0,
 7,
 11,
 12,
 4,
 7,
 14,
 4,
 4,
 0,
 4,
 10,
 6,
 7,
 7,
 5,
 12,
 6,
 6,
 7,
 7,
 6,
 5,
 7,
 12,
 7,
 7,
 6,
 0,
 6,
 5,
 14,
 5,
 2,
 4,
 1,
 12,
 7,
 9,
 7,
 6,
 6,
 7,
 5,
 6,
 7,
 3,
 15,
 4,
 0,
 7,
 7,
 2,
 15,
 13,
 12,
 4,
 14,
 6,
 4,
 6,
 7,
 2,
 12,
 6,
 0,
 6,
 0,
 7,
 7,
 13,
 14,
 4,
 7,
 12,
 7,
 14,
 6,
 7,
 7,
 7,
 4,
 6,
 7,
 4,
 7,
 5,
 7,
 7,
 5,
 6,
 5,
 4,
 14,
 7,
 4,
 4,
 6,
 7,
 4,
 6,
 6,
 6,
 6,
 7,
 14,
 6,
 7,
 6,
 6,
 6,
 13,
 5,
 7,
 6,
 4,
 5,
 6,
 4,
 4,
 6,
 4,
 4,
 6,
 15,
 5,
 7,
 5,
 10,
 14,
 14,
 4,
 12,
 5,
 12,
 14,
 6,
 7,
 14,
 5,
 4,
 7,
 7,
 4,
 3,
 15,
 3,
 6,
 12,
 12,
 5,
 5,
 7,
 1,
 6,
 4,
 4,
 7,
 4,
 7,
 5,
 4,
 15,
 4,
 6,
 8,
 14,
 9,
 1,
 4,
 5,
 5,
 7,
 4,
 6,
 7,
 15,
 12,
 12,
 5,
 12,
 6,
 7,
 13,
 14,
 6,
 7,
 7,
 7,
 7,
 0,
 6,
 15,
 6,
 7,
 4,
 5,
 7,
 14,
 7,
 13,
 4,
 4,
 12,
 1,
 7,
 5,
 7,
 7,
 15,
 6,
 4,
 7,
 6,
 4,
 5,
 15,
 3,
 4,
 4,
 14,
 7,
 6,
 6,
 6,
 7,
 6,
 5,
 7,
 15,
 5,

In [24]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
Y_train = torch.LongTensor(Y_train)
Y_test = torch.LongTensor(Y_test)

In [62]:
criterion  = nn.CrossEntropyLoss()
optimizer  = torch.optim.Adam(model.parameters(),lr=0.00001)

In [84]:
model.parameters

<bound method Module.parameters of MBTI_NN(
  (hidden1): Linear(in_features=4096, out_features=16, bias=True)
  (sigmoid): Sigmoid()
  (softmax): LogSoftmax()
)>

In [85]:
epochs = 100
losses = []
for i in range(epochs):
    y_pred = model.forward(X_train)
    loss = criterion(y_pred, Y_train)
    losses.append(loss)
    
    if i%2==0:
        print(f"epoch: {i} and loss {loss}")
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

epoch: 0 and loss 2.777109146118164
epoch: 2 and loss 2.777109146118164
epoch: 4 and loss 2.777109146118164
epoch: 6 and loss 2.777109146118164
epoch: 8 and loss 2.777109146118164
epoch: 10 and loss 2.777109146118164
epoch: 12 and loss 2.777109146118164
epoch: 14 and loss 2.777109146118164
epoch: 16 and loss 2.777109146118164
epoch: 18 and loss 2.777109146118164
epoch: 20 and loss 2.777109146118164
epoch: 22 and loss 2.777109146118164
epoch: 24 and loss 2.777109146118164
epoch: 26 and loss 2.777109146118164
epoch: 28 and loss 2.777109146118164
epoch: 30 and loss 2.777109146118164
epoch: 32 and loss 2.777109146118164
epoch: 34 and loss 2.777109146118164
epoch: 36 and loss 2.777109146118164
epoch: 38 and loss 2.777109146118164
epoch: 40 and loss 2.777109146118164
epoch: 42 and loss 2.777109146118164
epoch: 44 and loss 2.777109146118164
epoch: 46 and loss 2.777109146118164
epoch: 48 and loss 2.777109146118164
epoch: 50 and loss 2.777109146118164
epoch: 52 and loss 2.777109146118164
epoch:

In [48]:
from sklearn.linear_model import LogisticRegression

In [86]:
clf  = LogisticRegression(random_state=42,max_iter=1000,penalty='l2',C=0.1)
clf.fit(embedding, label_enc)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [87]:
clf.score(X_test, Y_test)

0.3254054054054054

In [70]:
from xgboost import XGBClassifier

m = XGBClassifier(
    max_depth=2,
    gamma=2,
    eta=0.8,
    reg_alpha=0.5,
    reg_lambda=0.5
)

In [72]:
m.fit(X_train, Y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.8, gamma=2,
              gpu_id=-1, importance_type='gain', interaction_constraints=None,
              learning_rate=0.800000012, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0.5,
              reg_lambda=0.5, scale_pos_weight=None, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None)

In [76]:
from sklearn.metrics import accuracy_score
y_pred = m.predict(X_test)
# evaluate predictions
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 23.46%


In [113]:
Y_mask = []
c_1 = 0
c_0 = 0
for i in label_enc:
    if i >=0 and i<=7:
        Y_mask.append(1)
        c_1+=1
    else:
        Y_mask.append(2)
        c_0+=1
print(c_0,c_1)

2033 7215


In [95]:
X_train_mask, X_test_mask, Y_train_mask, Y_test_mask = train_test_split(embedding,Y_mask,test_size=0.2,random_state=42)

In [127]:
clf  = LogisticRegression(random_state=42,max_iter=1000,penalty='l2',C=0.1)
clf.fit(X_train_mask, Y_train_mask)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [128]:
clf.score(X_test_mask, Y_test_mask)

0.6625086625086625

In [125]:
m = XGBClassifier(
    max_depth=2,
    gamma=2,
    eta=0.8,
    reg_alpha=0.5,
    reg_lambda=0.5
)
m.fit(X_train_mask, Y_train_mask)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.8, gamma=2,
              gpu_id=-1, importance_type='gain', interaction_constraints=None,
              learning_rate=0.800000012, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0.5,
              reg_lambda=0.5, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [126]:
from sklearn.metrics import accuracy_score
y_pred = m.predict(X_test_mask)
# evaluate predictions
accuracy = accuracy_score(Y_test_mask, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 80.70%


In [105]:
X_train_mask = torch.FloatTensor(X_train_mask)
X_test_mask = torch.FloatTensor(X_test_mask)
Y_train_mask = torch.LongTensor(Y_train_mask)
Y_test_mask = torch.LongTensor(Y_test_mask)

In [117]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_sm, y_sm = smote.fit_resample(embedding, Y_mask)

In [124]:
X_train_mask, X_test_mask, Y_train_mask, Y_test_mask = train_test_split(X_sm,y_sm,test_size=0.2,random_state=42)
X_train_mask = torch.FloatTensor(X_train_mask)
X_test_mask = torch.FloatTensor(X_test_mask)
Y_train_mask = torch.LongTensor(Y_train_mask)
Y_test_mask = torch.LongTensor(Y_test_mask)

In [129]:
clf  = LogisticRegression(random_state=42,max_iter=10000,penalty='l2',C=0.1)
clf.fit(X_train_mask, Y_train_mask)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [130]:
clf.score(X_test_mask, Y_test_mask)

0.6625086625086625